In [10]:
# import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pymysql
from sqlalchemy import create_engine
import getpass  # To get the password without showing the input
password = getpass.getpass()

In [11]:
#  transform exl format to csv
def remove_non_ascii(text):
    if isinstance(text, str):  # Check if the input is a string
        return ''.join(char for char in text if ord(char) < 128)
    return text  # Return as is if not a string


# Load your DataFrame from the Excel file
df = pd.read_excel('./db1.xlsx')

# Apply the function to all string columns in the DataFrame, making sure to handle non-string types
for column in df.select_dtypes(include=['object']).columns:
    # Convert to string before applying function
    df[column] = df[column].astype(str).apply(remove_non_ascii)

# Save the cleaned DataFrame to a new CSV file, ready for import into MySQL Workbench
df.to_csv('db1.csv', index=False)

print("Cleaned data has been saved to 'db1.csv'.")

Cleaned data has been saved to 'db1.csv'.


In [12]:
# Connect SQL to Python
user = "root"
password = "1234abcd"
host = "127.0.0.1"
port = "3306"
db = "bank_loan_project"
# create engine
engine = create_engine(
    f'mysql+mysqlconnector://{user}:{password}@{host}:{port}/{db}')

In [13]:
# KPI-1: loan_amount_per_year
kpi1_query = """
SELECT YEAR(issue_d) AS year_of_issue_d, SUM(loan_amnt) AS total_loan_amnt
FROM db1
GROUP BY year_of_issue_d
ORDER BY year_of_issue_d;
"""
kpi1 = pd.read_sql(kpi1_query, con=engine)
kpi1.to_csv('./results/1.loan_amount_per_year.csv', index=False)
kpi1.head()

,year_of_issue_d,total_loan_amnt
0,2007,2212100.0
1,2008,14085750.0
2,2009,45277975.0
3,2010,120965300.0
4,2011,42096200.0


In [14]:
# KPI-2: Revolving Balance by Grade and Subgrade
kpi2_query = """
SELECT grade, sub_grade, SUM(revol_bal) AS total_revol_bal
FROM db1 INNER JOIN db2
ON(db1.id = db2.id)
GROUP BY grade, sub_grade
ORDER BY grade, sub_grade;
"""
kpi2 = pd.read_sql(kpi2_query, con=engine)
kpi2.to_csv('./results/2.revol_bal_by_grade_and_subgrade.csv', index=False)
kpi2.head()

,grade,sub_grade,total_revol_bal
0,A,A1,2555122.0
1,A,A2,6155325.0
2,A,A3,11340243.0
3,A,A4,17115905.0
4,A,A5,22742484.0


In [15]:
# KPI-3: Total payment for Verified and Non Verified Status
kpi3_query = """
SELECT verification_status, ROUND(SUM(total_pymnt),2) AS total_payment
FROM db1 INNER JOIN db2
ON(db1.id = db2.id)
GROUP BY verification_status;
"""
kpi3 = pd.read_sql(kpi3_query, con=engine)
kpi3.to_csv(
    './results/3.total_payment_for_verified_and_non_verified_status.csv', index=False)
kpi3.head()

,verification_status,total_payment
0,Verified,1.397143e+08
1,Not Verified,1.020477e+08


In [16]:
# KPI-4: Loan Status by State and Last Credit Pull Date
kpi4_query = """
SELECT addr_State, last_Credit_pull_D, loan_Status
FROM db1 INNER JOIN db2
ON(db1.id = db2.id)
GROUP BY addr_State, last_Credit_pull_D, loan_Status
ORDER BY last_Credit_pull_D; 
"""
kpi4 = pd.read_sql(kpi4_query, con=engine)
kpi4.to_csv(
    './results/4.loan_status_by_state_and_last_credit_pull_date.csv', index=False)
kpi4.head()

,addr_State,last_Credit_pull_D,loan_Status
0,CA,,Charged Off
1,CA,,Fully Paid
2,NY,2007-05-01,Fully Paid
3,IN,2007-06-01,Fully Paid
4,WI,2007-06-01,Fully Paid


In [17]:
# KPI-5: Home ownership vs last payments date status per Year
kpi5_query = """
SELECT
home_ownership,
last_pymnt_d,
ROUND(SUM(last_pymnt_amnt),2) AS total_payment
FROM db1 INNER JOIN db2 ON (db1.id = db2.id)
GROUP BY home_ownership, last_pymnt_d
ORDER BY last_pymnt_d DESC, home_ownership DESC;
"""
kpi5 = pd.read_sql(kpi5_query, con=engine)
kpi5.to_csv(
    './results/5.home_ownership_vs_last_payments_date_status.csv', index=False)
kpi5.head()

,home_ownership,last_pymnt_d,total_payment
0,RENT,2016-05-01,691.92
1,MORTGAGE,2016-05-01,563.91
2,RENT,2016-04-01,4210.77
3,OWN,2016-04-01,110.15
4,MORTGAGE,2016-04-01,5113.21
